## Introduction to Flask
![flask logo](http://flask.pocoo.org/static/logo/flask.png)


Flask is a fast, lightweight way to connect your Python scripts to a server. It's a simple and robust framework that can do small tasks (create a microblog, stand up a simple API) or complex ones (Pinterest's API, create a twitter clone).

Let's jump in with a simple example. Then, we'll expand it to show what it can do with your models. But first you may need to:

```bash
$ pip install Flask
```



## Hello, world.
Create a new file called `hello.py` . Type in this code line by line. No copy pasting!

```Python
import flask
app = flask.Flask(__name__)

@app.route("/")
def hello():
    return "Hello World!"

if __name__ == '__main__':
    app.run(debug=True)
```




Three things happen here:
- initialize the app
- use built-in decorators\* to define what happens on a page
- launch the app

\* I'll explain soon...

Now launch the file from your command line:

```bash
$ python hello.py
* Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
```

Go to that URL$^*$ in your browser to see your app running on your `localhost`. (Or use `curl`!)


$^*$ **Your port may not be 5000. There may also be a longer URL which you need to copy-paste exactly.**


### Arguments and styling

Add the following route underneath the hello() function:

```Python
@app.route('/greet/<name>')
def greet(name):
    '''Say hello to your first parameter'''
    return "Hello, {}!".format(name)
```

Save, and the app should automatically restart; if it doesn't, you can `ctrl-C` and launch it again yourself. Now navigate to `http://127.0.0.1:5000/greet/Roger`. Your function should respond to that input!



### Decorators

What's a decorator? The short story: they put wrap one function around another. So the previous function is actually:

```Python
app.route(greet(name))
```

See the Further Reading section for a full tutorial. For now, we just need to know that the `@app.route(endpoint)` pattern is how you tell Flask to listen to a particular URL, and what to do if requests are sent there.

Since the `return` statement sends text to an HTML page, you can style it with HTML tags:

```Python
@app.route("/")
def hello():
    return '''
    <body>
    <h2> Hello World! <h2>
    </body>
    '''
```

(If you make any coding mistakes, your server may shut down with an error message. Fix the code and rerun!)



## Add in machine learning
We can use Flask to share our machine learning predictions.

Create a new file `application.py`. Import and initialize the flask app, and launch the server at the bottom. Leave room in the middle to add in your model and routes later on.




```Python
import flask
app = flask.Flask(__name__)

#-------- MODEL GOES HERE -----------#

#-------- ROUTES GO HERE -----------#

if __name__ == '__main__':
    '''Connects to the server'''

    HOST = '127.0.0.1'
    PORT = '4000'
    app.run(HOST, PORT)
```

Note that this time we specifed the host and port we want the app to run on.


### Deploying an SMS spam detector

Here's an idea for a first ML app: something to identify text message spam.

You already know how to make this! Let's hook it up to an endpoint.

In [5]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

In [6]:
df = pd.read_csv('data/SMSSpamCollection.txt', sep='\t', header=None)
df.columns = ['target', 'msg']
y = df['target']
X = df['msg']

In [7]:
cvec = TfidfVectorizer(stop_words='english', max_features = 300)
X = cvec.fit_transform(X)
clf = MultinomialNB()
clf.fit(X, y)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [14]:
text = 'Can I interest you in a new blender? I need your credit card number and social security number.'
msg = pd.Series(text)
X_new = cvec.transform(msg)
score = clf.predict(X_new)

In [15]:
score

array(['ham'], dtype='<U4')

Import the libraries we need, load our SMS spam dataset from the NLP lesson, and define our target and feature variables.

```Python

#-------- MODEL GOES HERE -----------#
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

df = pd.read_csv('data/SMSSpamCollection.txt', sep='\t', header=None)
df.columns = ['target', 'msg']
y = df['target']
X = df['msg']
```

Now vectorize the features (with help from TfidfVectorizer) and fit a Naive Bayes model.

```Python
# Tfidf, filter stop words, 300 features
cvec = TfidfVectorizer(stop_words='english', max_features = 300)
X = cvec.fit_transform(X)
clf = MultinomialNB()
clf.fit(X, y)
```


### Make a simple API
Here's the fun part. Now that we have a classifier, we need to get some values to make our predictions.

One way to do this is to get information from the **URL parameters**. These are the part of a URL that come after the `?` and are matched by key:value pairs. For example, if you navigate to
`http://localhost:4000/is_spam?msg="50% off fidget spinners!"` Flask can retrieve that message data for you. Let's write a route to do just that:


This will look for a `GET` request containing a `msg` key at the `/is_spam` endpoint.

So visiting `http://localhost:4000/is_spam?msg="50% off fidget spinners!"` will set `flask.request.args['msg']` to "50% off fidget spinners!". Then we transform this into something the model can read.

```Python
#-------- ROUTES GO HERE -----------#
@app.route('/is_spam', methods=["GET"])
def is_spam():
    msg = pd.Series(flask.request.args['msg'])
    X_new = cvec.transform(msg)
    score = clf.predict(X_new)
    results = {'prediction': score[0]}
    return flask.jsonify(results)
```

And... voila! Save the file. Launch your app. (Give it 5-10 seconds to start.) You now have a simple API for your model. Try visiting:

`http://localhost:4000/is_spam?msg="50% off fidget spinners!"`

Or

`http://localhost:4000/is_spam?msg="pick up milk at the store"`

Play with the message in the URL. You should get a single prediction of spam or ham in neatly formatted JSON. (`localhost` is just a variable set to 127.0.0.1 by default.)


### Problems

1. Build a new Flask application using our `pitchfork.json` data that takes in a genre and returns the top 10 reviews for the genre.

2. Add a second field to take in a year to search by.  Demonstrate your application works by finding the top 10 electronic records of 2013.

# Further reading

### Flask

- [The Flask Documentation](http://flask.pocoo.org/docs/0.11/)
- [A Flask tutorial to follow along with](https://github.com/miguelgrinberg/flask-pycon2014)
- [Another tutorial that gets into CSS styling](https://code.tutsplus.com/tutorials/an-introduction-to-pythons-flask-framework--net-28822)
- [The Flask mega tutorial](http://blog.miguelgrinberg.com/post/the-flask-mega-tutorial-part-ii-templates)
- [Flask's development server is not for production](https://stackoverflow.com/questions/12269537/is-the-server-bundled-with-flask-safe-to-use-in-production)
- [Setting up Flask on AWS EC2](http://bathompso.com/blog/Flask-AWS-Setup/). This should be your next step if you want to share your model with the world!
- [A great guide to those weird "decorators"](http://simeonfranklin.com/blog/2012/jul/1/python-decorators-in-12-steps/).

### Production coding

- Add [logging](https://fangpenlin.com/posts/2012/08/26/good-logging-practice-in-python/) to your code; you'll be very glad you did.
- Think ahead and include [error handling](https://eli.thegreenplace.net/2008/08/21/robust-exception-handling/), via [try/except clauses](https://jeffknupp.com/blog/2013/02/06/write-cleaner-python-use-exceptions/)
- Get more comfortable with git, including [feature branching](https://www.atlassian.com/git/tutorials/comparing-workflows/feature-branch-workflow).
- Include [unit tests](http://www.diveintopython.net/unit_testing/index.html); the [pytest module](http://pythontesting.net/framework/pytest/pytest-introduction/) is great.
- [Integrate databases](http://zetcode.com/db/sqlitepythontutorial/)!
- Beware technical debt, especially [machine learning technical debt](https://static.googleusercontent.com/media/research.google.com/en//pubs/archive/43146.pdf).